In [30]:
import pandas as pd

# (1) Sas File Load

In [31]:
sas_file1=r"C:\Users\USER\Desktop\비만 예측 REMASTER\data\2008_골밀도_체지방.sas7bdat"
data_08_fat=pd.read_sas(sas_file1)

sas_file2=r"C:\Users\USER\Desktop\비만 예측 REMASTER\data\2008_기본.sas7bdat"
data_08_base=pd.read_sas(sas_file2)

sas_file3=r"C:\Users\USER\Desktop\비만 예측 REMASTER\data\2009_골밀도_체지방.sas7bdat"
data_09_fat=pd.read_sas(sas_file3)

sas_file4=r"C:\Users\USER\Desktop\비만 예측 REMASTER\data\2009_기본.sas7bdat"
data_09_base=pd.read_sas(sas_file4)


# (2) 년도별로 기본 DB와 체지방DB를 병합

In [32]:
data_08_base.rename(columns={'id': 'ID'}, inplace=True)
data_09_base.rename(columns={'id': 'ID'}, inplace=True)

In [33]:
data_08=\
    pd.merge(data_08_base, data_08_fat, on='ID', how='left')

In [34]:
data_09=\
    pd.merge(data_09_base, data_09_fat, on='ID', how='left')

# (3) 08년도 데이터와 09년도 데이터 병합

In [35]:
data_08_09=pd.concat([data_08, data_09], axis=0, join='inner')
data_08_09.columns = data_08_09.columns.str.replace('_x$', '', regex=True)
#정규식 사용해서 join에서 생기는 _x를 없앰

# (4) TATGET인 'fat' 특성을 만들기 위해 필요한 DW_WBT_pFT (체지방률)이 결측치인 행 삭제

In [36]:
data=data_08_09

In [37]:
data=data.dropna(subset=['DW_WBT_pFT'])

- 기존 프로젝트에서 1차로 주관적으로 선택했던 특성(기본 정보(나이, 소득분위...), 활동량 정보(걸음수 등).. ) 124개와 같은 특성만 남김

In [38]:
last_col=['sex', 'age', 'incm5', 'ho_incm5', 'edu', 'occp', 'genertn', 'allownc', 'house', 'marri_1', 'D_1_1', 'DI1_dg', 'DI1_2', 'DI2_dg', 'DI2_2', 'DI3_dg', 'DI4_dg', 'DI8_dg', 'DM1_dg', 'DM4_dg', 'DM6_dg', 'DJ4_dg', 'DJ4_3', 'DJ5_dg', 'DJ6_dg', 'DJ8_dg', 'DF2_dg', 'DL2_dg', 'DL1_dg', 'DN1_dg', 'DN2_dg', 'DK7_dg', 'DE1_dg', 'DE1_32', 'DE2_dg', 'DH4_dg', 'DC1_dg', 'DC2_dg', 'DC3_dg', 'DC4_dg', 'DC5_dg', 'DC6_dg', 'DK2_dg', 'DK8_dg', 'DK9_dg', 'DK4_dg', 'BH1', 'LQ4_00', 'LQ_1EQL', 'LQ_2EQL', 'LQ_3EQL', 'LQ_4EQL', 'LQ_5EQL', 'EC1_1', 'EC_ind', 'EC_occp', 'EC_lgw_2', 'EC_pjob_1', 'EC_pjob_4', 'EC_wht_1', 'EC_wht_2', 'EC_wht_3', 'EC_wht_4', 'EC_wht_5', 'EC_env_3', 'EC_env_4', 'EC_env_5', 'EC_env_6', 'EC_env_7', 'EC_env_8', 'BO1_1', 'BO1_3', 'BO2_1', 'BO3_01', 'BO3_02', 'BO3_03', 'BO3_05', 'BO3_04', 'BO3_12', 'BO3_07', 'BO3_09', 'BD1_11', 'BD2_1', 'BP8', 'BP1', 'BS3_1', 'BS5_21', 'BS5_22', 'LF_SAFE', 'LK_EDU', 'LK_LB_IT', 'LK_LAB', 'LK_GD', 'LS_FUN', 'L_OUT_FQ', 'L_SNA_FQ', 'N_DIET', 'N_INTK', 'N_EN', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO', 'LK_GD_P1', 'LK_GD_P2', 'LK_GD_P3', 'LK_GD_P4', 'LK_GD_P5', 'LK_GD_P6', 'LK_GD_P7', 'LS_VIT', 'L_BR_FAM', 'L_LN_FAM', 'L_DN_FAM', 'HE_ht', 'HE_wt', 'HE_wc', 'pa_mid', 'pa_walk', 'BE5_1', 'sm_presnt', 'BS8_2', 'BS9_2', 'DW_WBT_pFT', 'DW_WBT_BMD', 'fat', 'BMI']

In [39]:
common_col=data.columns.intersection(last_col)
data=data[common_col]

# (5) target값 생성 ('fat')

성별에 따른 체지방률('DW_WBT_pFT') 기준표

**남자**

- 정상: 16.7 ~ 22.5
- 과체중: 22.5 ~ 24.8
- 비만: 24.8 이상

**여자**

- 정상: 28.0 ~ 34.3
- 과체중: 34.3 ~ 36.49
- 비만: 36.5 이상


---
체지방률을 기준으로 한 target 특성인 'fat' 컬럼을 추가

- 0: 정상
- 1: 과체중
- 2: 비만


In [40]:
data['fat']=0
# 남자
data.loc[ (data['sex']==1) & (data['DW_WBT_pFT']<22.5), 'fat']=0  # 정상
data.loc[ (data['sex']==1) & (data['DW_WBT_pFT']>=22.5), 'fat']=1 # 과체중
data.loc[ (data['sex']==1) & (data['DW_WBT_pFT']>=24.8), 'fat']=2 # 비만

# 여자
data.loc[ (data['sex']==2) & (data['DW_WBT_pFT']<28.0), 'fat']=0
data.loc[ (data['sex']==2) & (data['DW_WBT_pFT']>=34.3), 'fat']=1
data.loc[ (data['sex']==2) & (data['DW_WBT_pFT']>=36.5), 'fat']=2

In [41]:
data.to_csv("data_set.csv", index=False)